# Simulator for Idealized Polymer Conformations

In [3]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import regex as re
from itertools import cycle
import math
import ipywidgets as widgets

bond_lengths = pd.read_csv("Bond Lengths.csv", index_col=0)

In [4]:
def fjc(blengths_cycle, n_mon, bpm):
    """Generate arrays of x, y, z coordinates under the freely jointed chain model
    Parameters: 
    blengths (cycle): cycle containing length of each bond in each repeat unit
    n_mon (int): number of repeat units in chain
    bpm (int): number of bonds in each repeat unit
    
    Returns:
    x, y, z (arrays): arrays containing coordinates for each atom in the polymer chain
    """
    x, y, z = np.zeros((1, 1)), np.zeros((1, 1)), np.zeros((1, 1))
    for i in np.arange(n_mon*bpm):
        prev_x, prev_y, prev_z = x[i], y[i], z[i]
        blength = next(blengths_cycle)
        assert not math.isnan(blength), "No data for bond type"
        new_x = np.random.choice(np.linspace(-blength, blength, 1000))
        new_y = np.random.choice(np.linspace(-np.sqrt(blength**2-new_x**2), np.sqrt(blength**2-new_x**2), 1000))
        new_z = np.random.choice([-1, 1])*np.sqrt(np.round((blength**2)-(new_x**2)-(new_y**2), 5))
        x, y, z = np.append(x, prev_x+new_x), np.append(y, prev_y+new_y), np.append(z, prev_z+new_z) 
    return x, y, z

def frc(blengths_cycle, n_mon, bpm, bangle):
    """Generate arrays of x, y, z coordinates under the freely rotating chain model
    Parameters: 
    blengths (cycle): cycle containing length of each bond in each repeat unit
    n_mon (int): number of repeat units in chain
    bpm (int): number of bonds in each repeat unit
    bangle (int): bond angle between successive backbone species
    
    Returns:
    x, y, z (arrays): arrays containing coordinates for each atom in the polymer chain
    """
    half_rad = np.pi - bangle*np.pi/180
    x, y, z = fjc(blengths_cycle, 1, 1)
    for i in np.arange(1, n_mon*bpm-1):
        blength = next(blengths_cycle)
        assert not math.isnan(blength), "No data for bond type"
        
        prev_x, prev_y, prev_z = x[i], y[i], z[i]
        twoprev_x, twoprev_y, twoprev_z = x[i-1], y[i-1], z[i-1]
        
        x_length = abs(blength*np.cos(half_rad))
        
        x_vec = np.array([prev_x - twoprev_x, prev_y - twoprev_y, prev_z - twoprev_z])
        x_vec /= np.sqrt(np.sum(x_vec**2))
        y_vec = np.array([-1/x_vec[0], 1/x_vec[1], 0])
        y_vec /= np.sqrt(np.sum(y_vec**2))
        z_vec = np.cross(x_vec, y_vec)
        z_vec /= np.sqrt(np.sum(z_vec**2))
        
        x_proj =  x_vec*x_length
        yz_length = abs(blength*np.sin(half_rad))
        y_length = np.random.choice(np.linspace(-yz_length, yz_length, 10000))
        y_proj = y_length*y_vec
        z_length = np.random.choice([-1, 1])*np.sqrt(yz_length**2 - y_length**2)
        z_proj =  z_length*z_vec
        new_vec = x_proj + y_proj + z_proj
        x, y, z = np.append(x, prev_x + new_vec[0]), np.append(y, prev_y + new_vec[1]), np.append(z, prev_z + new_vec[2])
    return x, y, z

def ris(blengths_cycle, n_mon, bpm, bangle):
    """Generate arrays of x, y, z coordinates under the isometric rotation chain model
    Parameters: 
    blengths (cycle): cycle containing length of each bond in each repeat unit
    n_mon (int): number of repeat units in chain
    bpm (int): number of bonds in each repeat unit
    bangle (int): bond angle between successive backbone species
    
    Returns:
    x, y, z (arrays): arrays containing coordinates for each atom in the polymer chain
    """
    x, y, z = frc(blengths_cycle, 1, 3, bangle)
    for i in np.arange(2, n_mon*bpm):
        blength = next(blengths_cycle)
        axis = np.array([x[i]-x[i-1], y[i]-y[i-1], z[i]-z[i-1]])
        axis /= np.sqrt(np.sum(axis**2))
        vec = np.array([x[i-1]-x[i-2], y[i-1]-y[i-2], z[i-1]-z[i-2]])
        vec /= np.sqrt(np.sum(vec**2))
        phi = np.random.choice([0, 2*np.pi/3, 4*np.pi/3], p=[0.5, 0.25, 0.25])
        
        new_vec = np.cos(phi)*vec + (1-np.cos(phi))*(vec@axis)*axis + np.sin(phi)*np.cross(vec, axis)
        
        x, y, z = np.append(x, x[i] + new_vec[0]), np.append(y, y[i] + new_vec[1]), np.append(z, z[i] + new_vec[2])
        
    return x, y, z

In [5]:
def simulate_conformation(model, n_mon, chain_formula, bangle=0, params=False):
    """Process chain formula to identify bond identities and generate arrays of x, y, z coordinates using a call to fjc, frc, or ris
    Parameters:
    model (str): theoretical model to use when generating conformation
    n_mon (int): number of repeat units in chain
    chain_formula (str): chemical formula of polymer chain
    params (bool): bool value corresponding to whether or not bonds per monomer and bond lengths should be returned
    
    Returns:
    x_coords, y_coords, z_coords (arrays): arrays containing coordinates for each atom in the polymer chain
        If params passed as True:
        bpm (int): bonds per monomer
        blengths (list): lengths of bonds in repeat unit
    """
    assert model in ["fjc", "frc", "ris"], "Invalid conformational model"
    elements = re.findall(r"[A-Z][^A-Z]*", chain_formula)
    for element in elements:
        assert element in bond_lengths.index, "Invalid element in polymer backbone"
    bonds = []
    for i in np.arange(len(elements)-1):
        bonds.append([elements[i], elements[i+1]])
    bonds.append([elements[0], elements[-1]])
    bpm = len(bonds)
    blengths = [bond_lengths.loc[bond[0], bond[1]] for bond in bonds]
    blengths_cycle = cycle(blengths)
    if model=="fjc":
        x_coords, y_coords, z_coords = fjc(blengths_cycle, n_mon, bpm)
    elif model=="frc":
        x_coords, y_coords, z_coords = frc(blengths_cycle, n_mon, bpm, bangle)
    elif model=="ris":
        x_coords, y_coords, z_coords = ris(blengths_cycle, n_mon, bpm, bangle)
    if params:
        return x_coords, y_coords, z_coords, bpm, blengths
    return x_coords, y_coords, z_coords

**Modeling single-chain conformational entropy and ideal free energy**

In [6]:
def plot_conformation(model, n_mon, chain_formula, bangle):
    """Generate 3D plot of stimulated conformation annotated with end-to-end distance and entropic spring constant
    Parameters:
    model (str): theoretical model to use when generating conformation 
    n_mon (int): number of repeat units in chain
    chain_formula (str): chemical formula of polymer chain (eg "CC", "SiO", etc)
    
    Returns:
    None
    """
    x_coords, y_coords, z_coords, bpm, blengths = simulate_conformation(model, n_mon, chain_formula, bangle, True)
    R2 = round(x_coords[-1]**2 + y_coords[-1]**2 + z_coords[-1]**2, 2)
    k = ((3*(1.381*10**-23)*298)/((n_mon*bpm)*np.mean(list(blengths))))
    conformation = go.Figure(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, marker={"size":0.1}, line={"width":2}, name="Simulated chain"))
    conformation.add_trace(go.Scatter3d(x=[0, x_coords[-1]], y=[0, y_coords[-1]], z=[0, z_coords[-1]], marker={"size":0.1}, line={"dash":"dash", "width":2}, name="End-to-end distance"))
    conformation.update_layout(scene={"xaxis":{"backgroundcolor":"white", "visible":False}, "yaxis":{"backgroundcolor":"white", "visible":False}, "zaxis":{"backgroundcolor":"white", "visible":False}}, 
                               title="<b> {} chain simulation under {} model with degree of polymerization {} </b> <br> Squared end-to-end distance: {} Å <br> Entropic spring constant at 298 K: {:.2E} N/Å".format(chain_formula, model.upper(), n_mon, R2, k))
    conformation.show()

In [7]:
#Create widgets for user interaction
plot_output = widgets.Output()

model = widgets.Dropdown(options=[("Freely Jointed Chain","fjc"), ("Freely Rotating Chain", "frc"), ("Rotational Isometric State", "ris")], description="Chain Model:", layout=widgets.Layout(width='25%'))
plot_dp = widgets.IntText(description='Degree Poly:', disabled=False, layout=widgets.Layout(width='15%'))
plot_chain_formula = widgets.Text(placeholder='Chemical formula of backbone (eg CC, SiO)', description='Backbone:', disabled=False, layout=widgets.Layout(width='40%'))
plot_bangle = widgets.FloatText(description='Bond Angle', disabled=False, layout=widgets.Layout(width='15%'))
plot_bangle.layout.visibility = 'hidden'

def run_plot_sim(b):
    """Generate single-chain conformation plot when widget button is pressed
    """
    with plot_output:
        plot_output.clear_output()
        plot_conformation(model.value, plot_dp.value, plot_chain_formula.value, plot_bangle.value)

plot_button = widgets.Button(description='Simulate', disabled=False, button_style='', tooltip='Click me')
plot_button.on_click(run_plot_sim)

plot_inputs = widgets.HBox([model, plot_dp, plot_chain_formula])

def show_angle(mod):
    if mod.new != "fjc":
        plot_bangle.layout.visibility = 'visible'
    else:
        plot_bangle.layout.visibility = 'hidden'
    

model.observe(show_angle, names="value") 

plot_inputs = widgets.HBox([model, plot_dp, plot_chain_formula, plot_bangle])

## Simulation of single-chain conformation and entropic spring constant

In [8]:
display(plot_inputs)
display(plot_button)
display(plot_output)

Button(description='Simulate', style=ButtonStyle(), tooltip='Click me')

Output()

In [9]:
def compare_models(n_mon, chain_formula, bangle=0):
    """Generate side-by-side 3D simulations comparing three theoretical conformation models
    Parameters: 
    n_mon (int): number of repeat units in chain
    chain_formula (str): chemical formula of polymer chain (eg "CC", "SiO", etc)
    
    Returns:
    None
    """
    fig = make_subplots(cols = 3, rows=1, 
                        specs=[[{"type":"scatter3d"}, {"type":"scatter3d"}, {"type":"scatter3d"}]], 
                        subplot_titles=["Freely Jointed Chain Model", "Freely Rotating Chain Model", "Rotational Isometric State Model"])

    fjc_x, fjc_y, fjc_z = simulate_conformation("fjc", n_mon, chain_formula)
    frc_x, frc_y, frc_z = simulate_conformation("frc", n_mon, chain_formula, bangle)
    ris_x, ris_y, ris_z = simulate_conformation("ris", n_mon, chain_formula, bangle) #change to hrc
    
    fig.add_trace(go.Scatter3d(x=fjc_x, y=fjc_y, z=fjc_z, marker={"size":0.1}, line={"width":2}), col=1, row=1)
    fig.add_trace(go.Scatter3d(x=frc_x, y=frc_y, z=frc_z, marker={"size":0.1}, line={"width":2}), col=2, row=1)
    fig.add_trace(go.Scatter3d(x=ris_x, y=ris_y, z=ris_z, marker={"size":0.1}, line={"width":2}), col=3, row=1) 

    fw = go.FigureWidget(fig)
    fw.layout.title="<b> {} chain simulation with degree of polymerization {} <b>".format(chain_formula, n_mon)
    fw.layout.showlegend=False
    fw.layout.scene1={"xaxis":{"backgroundcolor":"white", "visible":False}, "yaxis":{"backgroundcolor":"white", "visible":False}, "zaxis":{"backgroundcolor":"white", "visible":False}}
    fw.layout.scene2={"xaxis":{"backgroundcolor":"white", "visible":False}, "yaxis":{"backgroundcolor":"white", "visible":False}, "zaxis":{"backgroundcolor":"white", "visible":False}}
    fw.layout.scene3={"xaxis":{"backgroundcolor":"white", "visible":False}, "yaxis":{"backgroundcolor":"white", "visible":False}, "zaxis":{"backgroundcolor":"white", "visible":False}}
    fw.show()

In [10]:
#Create widgets for user interaction
compare_output = widgets.Output()

compare_dp = widgets.IntText(description='Degree Poly:', disabled=False, layout=widgets.Layout(width='20%'))
compare_chain_formula = widgets.Text(placeholder='Chemical formula of backbone (eg CC, SiO)', description='Backbone:', disabled=False, layout=widgets.Layout(width='40%'))
compare_bangle = widgets.FloatText(description='Bond Angle', disabled=False, layout=widgets.Layout(width='20%'))

def run_compare_sim(b):
    """Generate side-by-side comparison plots when widget button is pressed
    """
    with compare_output:
        compare_output.clear_output()
        compare_models(compare_dp.value, compare_chain_formula.value, compare_bangle.value)
    
compare_button = widgets.Button(description='Simulate', disabled=False, button_style='', tooltip='Click me')
compare_button.on_click(run_compare_sim)

compare_inputs = widgets.HBox([compare_dp, compare_chain_formula, compare_bangle])

## Comparison of conformational models

In [11]:
display(compare_inputs)
display(compare_button)
display(compare_output)

Button(description='Simulate', style=ButtonStyle(), tooltip='Click me')

Output()